<a href="https://colab.research.google.com/github/rohit10tiwari/MNIST_Fashion_CNN_Optimized/blob/master/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Create a CNN model and optimise it using Keras tuner**

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=21df45b33e503fdd21f0793df814e03c4818a339539036d9d05ff128d37ade3a
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=e2dbbc4a6a73134da93f313d841a508b3a74540cc6ca21a49a74a648b762c402
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.2.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)


In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),                        
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')    
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy', 
                          max_trials=5, directory= 'output', project_name='Mnist Fashion')

In [12]:
tuner_search.search(train_images,train_labels,epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4833 - accuracy: 0.8317 - val_loss: 0.3971 - val_accuracy: 0.8567
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3512 - accuracy: 0.8715 - val_loss: 0.3359 - val_accuracy: 0.8758
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3152 - accuracy: 0.8843 - val_loss: 0.3435 - val_accuracy: 0.8762


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4185 - accuracy: 0.8497 - val_loss: 0.3089 - val_accuracy: 0.8835
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2759 - accuracy: 0.9000 - val_loss: 0.2851 - val_accuracy: 0.8973
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2237 - accuracy: 0.9166 - val_loss: 0.2485 - val_accuracy: 0.9067


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3813 - accuracy: 0.8626 - val_loss: 0.2839 - val_accuracy: 0.8938
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2368 - accuracy: 0.9127 - val_loss: 0.2693 - val_accuracy: 0.9102
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1776 - accuracy: 0.9329 - val_loss: 0.2789 - val_accuracy: 0.9090


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4925 - accuracy: 0.8224 - val_loss: 0.4392 - val_accuracy: 0.8450
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3632 - accuracy: 0.8660 - val_loss: 0.3546 - val_accuracy: 0.8643
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3420 - accuracy: 0.8748 - val_loss: 0.3472 - val_accuracy: 0.8745


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5389 - accuracy: 0.8133 - val_loss: 0.4685 - val_accuracy: 0.8330
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4032 - accuracy: 0.8509 - val_loss: 0.4164 - val_accuracy: 0.8503
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3596 - accuracy: 0.8676 - val_loss: 0.3750 - val_accuracy: 0.8612


INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [1]:
model.summary()

NameError: ignored